## Extract Wiki Data and load data into DataFrame

In [2]:
import requests
from bs4 import BeautifulSoup
import html5lib
import pandas as pd
from geopy.geocoders import Nominatim
#import geocoder
import folium
import json
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as  cm
import matplotlib.colors as colors
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data=requests.get(url).text
soup=BeautifulSoup(data,'html5lib')
#print(soup)
tb_data=soup.find('table',attrs={"class":"wikitable sortable"})
#print(tb_data)
columns=[]
for i in tb_data.find_all('th'):
    columns.append(i.text.strip())
#print(columns)
df=pd.DataFrame(columns=columns)
for i in tb_data.find_all('tr')[1:]:
    l=i.text.replace('\n\n','\n').strip().split('\n')
    postal_code=l[0]
    borough=l[1]
    neighborhood=l[2]
    if borough!='Not assigned' and neighborhood=='Not assigned':
        neighborhood='borough'
    if borough !='Not assigned':
        df=df.append({'Postal Code':postal_code,
                   'Borough':borough,
                   'Neighbourhood':neighborhood},ignore_index=True)
df1=pd.read_csv('c:\\users\\nagab\\downloads\\Geospatial_Coordinates.csv')
df_final=pd.concat([df,df1[['Latitude','Longitude']]],axis=1)
df_final=df.join(df1.set_index('Postal Code'),on='Postal Code')
address='Toronto ,Canada'
geoloc=Nominatim(user_agent='tr_explorer')
location=geoloc.geocode(address)
tc_latitude=location.latitude
tc_longitude=location.longitude

df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [39]:
df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


##  Visualization Of All Neighborhoods in Toronto,Canada

In [33]:
t_map=folium.Map(location=[tc_latitude,tc_longitude],zoom_start=10)
# add all neighbors to map
for lat,lng,poscode,neigh in zip(df_final['Latitude'],df_final['Longitude'],df_final['Postal Code'],df_final['Neighbourhood']):
    label='{},{}'.format(poscode,neigh)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
                        [lat,lng],
                        radius=5,
                        color='red',
                        popup=label,
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.7
                        ).add_to(t_map)
t_map

## Explore the Venues under the North York ,Toronto , Canada Location

In [25]:
CLIENT_ID='BIF15ZNCC3JWA2NX1YM2BC1F4AIWTRIVJB5MA005V5BIRL4T'
CLIENT_SECRET='HIRYO04UTZBELNARPCH5QMXBI30Q5AVUVFCRZ1WJCIK4EYLT'
VERSION='20180604'
radius=500
limit=100

# analysis North York ,Toronto ,Canada data
north_york_toronto=df_final[df_final['Borough']==df_final.loc[0,'Borough']]
address='Downtown ,Toronto,Canada'
geoloc=Nominatim(user_agent='nt_explorer')
location=geoloc.geocode(address)
latitude=location.latitude
longitude=location.longitude
#create map.
n_map=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat,lng,poscode,neigh in zip(north_york_toronto['Latitude'],north_york_toronto['Longitude'],north_york_toronto['Postal Code'],north_york_toronto['Neighbourhood']):
    label='{},{}'.format(poscode,neigh)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
                        [lat,lng],
                        radius=5,
                        color='orange',
                        popup=label,
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.7
                        ).add_to(n_map)

n_map

##  Get Venue Categories and Venues likes count

In [44]:
#now analyze  the first row data in North York,Toronto Canada.
#get category for only one neighborhood
north_york_neighbor=north_york_toronto.loc[0,'Neighbourhood']
north_york_latitude=north_york_toronto.loc[0,'Latitude']
north_york_longitude=north_york_toronto.loc[0,'Longitude']
print("Neighborhood={}\nLatitude={}\nLongitude={}".format(north_york_neighbor,north_york_latitude,north_york_longitude,VERSION,radius,limit))

# to analyze one venues likes
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,north_york_latitude,north_york_longitude,VERSION,radius,limit)
results=requests.get(url).json()
def get_venue_likes(row):
    try:
        likes_venue_name=row['venue.name']
        venue_likes=row['venue.id']
    except:
        print('no data found')
        
    return venue_likes


results=results['response']['groups'][0]['items']
res_likes=pd.json_normalize(results)
filtered_columns=['venue.id','venue.name','venue.location.lat','venue.location.lng']
res_likes=res_likes.loc[:,filtered_columns]
res_likes['venue.likes']=res_likes.apply(get_venue_likes,axis=1)
for i in res_likes['venue.id']:
    url='https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(i,CLIENT_ID,CLIENT_SECRET,north_york_latitude,north_york_longitude,VERSION,radius,limit)
    results=requests.get(url).json()['response']['likes']['count']
    #print(results)

#to explore the all neighborhoods data

def get_likes(names,latitude,longitude,radius=500):
    venues_likes=[]
    for name,lat,lng in zip(names,latitude,longitude):
        url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,lat,lng,VERSION,radius,limit)
        try:
            results=requests.get(url).json()['response']['groups'][0]['items']
            venues_likes.append([(name,
                                 lat,
                                 lng,
                                 v['venue']['id'],
                                 v['venue']['name'],
                                 v['venue']['location']['lat'],
                                 v['venue']['location']['lng'],
                                 v['venue']['categories'][0]['name']) for v in results])
        except:
            print()
    nearby_venues=pd.DataFrame([item   for  venue_list in venues_likes for item in venue_list])
    nearby_venues.columns=['Neighbourhood','Latitude','Longitude','Venue_id','Venue_name','Venue_loc_lat','Venue_loc_lng','Venue_categories']
    return nearby_venues
nearby_venues_list=get_likes(names=north_york_toronto['Neighbourhood'],latitude=north_york_toronto['Latitude'],longitude=north_york_toronto['Longitude'])
def get_like_count(name,venue_id,lat,lng,venue_name,venue_cat):
    likes_list=[]
    c=0
    for name,v_id,lat,lng,v_name,v_cat in zip(name,venue_id,lat,lng,venue_name,venue_cat):
        url='https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(v_id,CLIENT_ID,CLIENT_SECRET,lat,lng,VERSION,radius,limit)
        c+=1
        res=requests.get(url).json()['response']['likes']['count']
        likes_list.append([(
                            name,
                            v_id,
                            v_name,
                            v_cat,
                            lat,
                            lng,
                            res)])
        
    nearby_likes_list=pd.DataFrame([item for venue_list in likes_list for item in venue_list])    
    nearby_likes_list.columns=['Neighbourhood','Venue_id','Venue_name','Venue_categories','Latitude','Longitude','Likes']
    print(c,"neighbour")
    return nearby_likes_list
nearby_venues_count=get_like_count(name=nearby_venues_list['Neighbourhood'],venue_id=nearby_venues_list['Venue_id'],lat=nearby_venues_list['Latitude'],lng=nearby_venues_list['Longitude'],venue_name=nearby_venues_list['Venue_name'],venue_cat=nearby_venues_list['Venue_categories'])
nearby_onehot=pd.get_dummies(nearby_venues_list['Venue_categories'],prefix="",prefix_sep="")
nearby_onehot=pd.concat([nearby_venues_count,nearby_onehot],axis=1)
final_df=nearby_onehot.sort_values('Likes',ascending=False).reset_index(drop=True).head(10)
#print(final_df)

likes_map=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat,lng,neigh,ven_id,likes in zip(nearby_venues_count['Latitude'],nearby_venues_count['Longitude'],nearby_venues_count['Neighbourhood'],nearby_venues_count['Venue_id'],nearby_venues_count['Likes']):
    label='{},{},{}'.format(neigh,ven_id,likes)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
                         [lat,lng],
                         radius=5,
                         popup=label,
                         fill=True,
                         fill_color='cyan',
                         fill_opacity=0.7
                         ).add_to(likes_map)
    
likes_map
#some mostlikely venues under the same latitude and longitude

Neighborhood=Parkwoods
Latitude=43.7532586
Longitude=-79.3296565
240 neighbour


In [30]:


#top 10 liked venues
likes_top_map=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat,lng,neigh,ven_id,likes in zip(final_df['Latitude'],final_df['Longitude'],final_df['Neighbourhood'],final_df['Venue_id'],final_df['Likes']):
    label='{},{},{}'.format(neigh,ven_id,likes)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
                         [lat,lng],
                         radius=5,
                         popup=label,
                         fill=True,
                         fill_color='cyan',
                         fill_opacity=0.7
                         ).add_to(likes_top_map)

likes_top_map
#final_df['Latitude']

## Modeling (KMeans Clustring)

In [67]:

#cluster   
n=5
clu=final_df.drop(['Neighbourhood','Venue_id','Latitude','Longitude','Venue_id','Venue_name','Venue_categories'],axis=1)
km=KMeans(n_clusters=n,random_state=0).fit(clu)
print(km.labels_)

[1 2 2 0 0 0 4 4 3 3]


## examine The results

In [68]:
final_df.loc[final_df['Cluster Labels']==0,final_df.columns[list(range(1,5))+[7]]]

,Neighbourhood,Venue_id,Venue_name,Venue_categories,Likes
3,Don Mills,4b1014fdf964a520b06823e3,Real Canadian Superstore,Supermarket,97
4,"Fairview, Henry Farm, Oriole",4ae07b8bf964a5208c7f21e3,Apple Fairview,Electronics Store,93
5,"Willowdale, Willowdale East",4ae257cff964a520758d21e3,Loblaws,Grocery Store,90


In [69]:
final_df.loc[final_df['Cluster Labels']==1,final_df.columns[list(range(1,5))+[7]]]

,Neighbourhood,Venue_id,Venue_name,Venue_categories,Likes
0,"Fairview, Henry Farm, Oriole",4ada3af3f964a520482021e3,CF Fairview Mall,Shopping Mall,500


In [71]:
final_df.loc[final_df['Cluster Labels']==2,final_df.columns[list(range(1,5))+[7]]]

,Neighbourhood,Venue_id,Venue_name,Venue_categories,Likes
1,"Willowdale, Willowdale East",4ad4c062f964a520e2f720e3,Cineplex Cinemas,Movie Theater,172
2,"Fairview, Henry Farm, Oriole",4b9413a4f964a5204f6634e3,SilverCity,Movie Theater,166


In [72]:
final_df.loc[final_df['Cluster Labels']==3,final_df.columns[list(range(1,5))+[7]]]

,Neighbourhood,Venue_id,Venue_name,Venue_categories,Likes
8,"Willowdale, Willowdale East",544ad69a498eeb86b7a4a8d0,Kinton Ramen,Ramen Restaurant,60
9,"Willowdale, Willowdale East",4b315d90f964a520840525e3,Ajisen Ramen 味千ラーメン,Ramen Restaurant,53


In [73]:
final_df.loc[final_df['Cluster Labels']==4,final_df.columns[list(range(1,5))+[7]]]

,Neighbourhood,Venue_id,Venue_name,Venue_categories,Likes
6,"Willowdale, Willowdale East",4aedfeadf964a52005d121e3,Starbucks,Coffee Shop,85
7,"Fairview, Henry Farm, Oriole",4ae083d4f964a520d47f21e3,Moxie's Classic Grill,American Restaurant,75


In [56]:
nearby_venues_list

,Neighbourhood,Latitude,Longitude,Venue_id,Venue_name,Venue_loc_lat,Venue_loc_lng,Venue_categories
0,Parkwoods,43.753259,-79.329656,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,4cb11e2075ebb60cd1c4caad,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,4c633acb86b6be9a61268e34,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,4f3ecce6e4b0587016b6f30d,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,4bbe904a85fbb713420d7167,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...,...
235,"Willowdale, Willowdale West",43.782736,-79.442259,4b352657f964a520c42a25e3,Tov-Li,43.784214,-79.446098,Pizza Place
236,"Willowdale, Willowdale West",43.782736,-79.442259,4bf818505efe2d7ff5186a34,Shoppers Drug Mart,43.784847,-79.446028,Pharmacy
237,"Willowdale, Willowdale West",43.782736,-79.442259,4bc75114af07a59340f07e2d,Tim Hortons,43.780940,-79.444231,Coffee Shop
238,"Willowdale, Willowdale West",43.782736,-79.442259,4b749c64f964a520a1e72de3,Price Chopper,43.783237,-79.446339,Grocery Store


In [74]:
final_df.head()

,Cluster Labels,Neighbourhood,Venue_id,Venue_name,Venue_categories,Latitude,Longitude,Likes,Accessories Store,Airport,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,1,"Fairview, Henry Farm, Oriole",4ada3af3f964a520482021e3,CF Fairview Mall,Shopping Mall,43.778517,-79.346556,500,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,"Willowdale, Willowdale East",4ad4c062f964a520e2f720e3,Cineplex Cinemas,Movie Theater,43.770120,-79.408493,172,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,"Fairview, Henry Farm, Oriole",4b9413a4f964a5204f6634e3,SilverCity,Movie Theater,43.778517,-79.346556,166,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,Don Mills,4b1014fdf964a520b06823e3,Real Canadian Superstore,Supermarket,43.725900,-79.340923,97,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,"Fairview, Henry Farm, Oriole",4ae07b8bf964a5208c7f21e3,Apple Fairview,Electronics Store,43.778517,-79.346556,93,0,0,...,0,0,0,0,0,0,0,0,0,0
